In [1]:
from pathlib import Path
import os.path
import os
import numpy as np
import re
import pandas as pd
import shutil


# create network 
1. get_node  
2. get_edge
3. get_network_fixed
    1. calls get_edge 
    2. get_node
4. the code calling get_net_work_fixed
5. add time stamp


In [7]:
soft_path = 'D:/mm_thesis/the_files/Data/'
csv_path =  'D:/mm_thesis/Software_graphs/D_graph/'
softwareList = os.listdir(soft_path)
soft_files = [(os.path.join(soft_path,infile)+"/") for infile in softwareList]
csv_path_l = os.listdir(csv_path)

In [8]:
soft_files

['D:/mm_thesis/the_files/Data/Cassandra/',
 'D:/mm_thesis/the_files/Data/Chukwa/',
 'D:/mm_thesis/the_files/Data/httpClient/',
 'D:/mm_thesis/the_files/Data/IVY/',
 'D:/mm_thesis/the_files/Data/Jena/']

### 1 get_node 

In [9]:
def get_node(node_line): 
        
    nodes = node_line.find("[") - 1 
    node_no = node_line[0:nodes]  
    n_end = node_line.find(",h")         # remove specail characters and skip the shape 
    node_name = node_line[nodes+2:n_end] 
    O_name = node_name 
    O_name = re.sub(r'(label=)','',O_name) 
    O_name = O_name.strip('"') 
    node_name =re.sub(r'(\\l)','',node_name) 
    node_name = re.sub(r'[\n \|| \\"]','',node_name) 
    node_name = re.sub(r'(label=)','',node_name) 
    #print(node_no, node_name) 
    args = " " 
    arg_st = node_name.find("<") 
    #print(node_name) 
    if node_name.rfind(".") != -1:   
        st_s = node_name.rfind(".") +1 

        if arg_st == -1:    # if node is a class   
            node_type = "class"
            class_c = node_name[st_s:]  
            class_p = node_name[0:st_s-1] 
        else: 
            node_type = 'method' #it's a method 
            #arg_st = node_name.find("<") 
            #print(arg_st) 
            #if arg_st != -1 : 

            arg_end = node_name.find(">")+1     
            #print(arg_end) 
            args = node_name[arg_st:arg_end] 
            args = args.strip('<') 
            args = args.strip('>') 

            st_s = node_name.rfind(".") +1 
            node_name = node_name[0: arg_st] 

            st_sm = node_name.rfind(".") 
            if st_sm != -1: 
                class_c = node_name[st_sm+1: arg_st] 
                class_p = node_name[0:st_sm] 
            else: 
                class_c = "" 
                class_p = "" 

               #print(node_name) 
               #print(type(args), 'from func') 
           #        args = np.array(args.split(',')) 
               
               #print(\"class_method\",args) 
                
    else : 
       #check if it has the colon identifier  
       #if arg_st != -1: 
       #print(node_name) 
        st_s =  len(node_name) + 1   #node is a java method so class and package are empty  
        class_c = " " 
        class_p = "" 
        node_type = "method"          
        #arg_st = node_name.find(\"<\") 
        #print(arg_st) 
        if arg_st != -1: 

            arg_end = node_name.find(">")+1 

            #print(arg_end) 
            args = node_name[arg_st:arg_end] 
            args = args.strip('<') 
            args = args.strip('>') 
            #print(type(args), 'from func') 
            #args = np.array(args.split(',')) 
            node_name= node_name[0:arg_st] 
            if node_name.rfind("::") != -1:   
                st_s = node_name.rfind("::") +2 
                class_c = node_name[st_s:]  
                class_p = node_name[0:st_s-1] 
                #print(class_c, class_p) 
                #print(\"java\",args) 
               #print(len(args)) 

            
    return (node_no,O_name,node_name, class_c, class_p,node_type, args)


### 2 get_edge

In [10]:
def get_edge(edge_line): 
    nedge= edge_line.find("->") 
    nsource = nedge -1 
    Node_s = edge_line[0:nsource] 
    source_node = Node_s 
    edgel = edge_line.find("[") 
    edgen = edge_line.find("]") 
    ntargetend = edgel - 1 
    ntarget = nedge +3 
    Node_t = edge_line[ntarget:ntargetend] 
    target_node = Node_t 
    edge_attr = edge_line[edgel+1:edgen -1] 
    dir_st = edge_attr.find("=") 
    dir_end= edge_attr.find(",") 
    edge_dir = edge_attr[dir_st+1:dir_end] 
    edge_dir = edge_dir.strip('"') 

       
       #print(\"s --> t\", Node_s, Node_t) 
        
       ### the direction is for vizualization  so this is not correct 
   #     if edge_dir == \"back\": 
   #             #print('dir back switch nodes') 
   #         source_node  = Node_t 
   #         target_node = Node_s  
               #print(\"s --> t\", source_node, target_node) 
          
                   
    # get the edge attributes, color , style, label  
            
    edge_c= edge_attr.find("color") 
    edge_ce = edge_attr[edge_c:].find(",") 
    color_len= edge_c + edge_ce 
    edge_color = edge_attr[edge_c+ 6:color_len].strip('"') 

    edge_sl = edge_attr.find("style") 
    edge_se = edge_attr[edge_sl:].find(",") 
    style_len= edge_sl + edge_se 
    edge_style = edge_attr[edge_sl+ 6:style_len].strip('"') 

    edge_la = edge_attr.find("label") 
    edge_label = " " 

    if edge_la != -1: 
        edge_le = edge_attr[edge_la:].find(",") 
        label_len= edge_la + edge_le 
        edge_label = edge_attr[edge_la + 6:label_len] 
        edge_label = edge_label.strip(' ') 
        edge_label = edge_label.strip('"') 

    return (source_node, target_node, edge_attr, edge_color, edge_style, edge_label)


## 3 get_net_work_fixed called get_edge and get_node

In [11]:
def get_network_fixed(dot_path, csv_graph,csv_node,ver): 
    fileList = os.listdir(dot_path) 
    dot_files = [os.path.join(dot_path,infile )for infile in fileList] 

    column_names = ['Nsource','Ntarget','edge_attr', 'e_color', 'e_style', 'e_label', 'source', 'target','ver'] 
    #umn_names = ['source', 'target','edge_attr','Nsource','Ntarget'] 
    all_edges = pd.DataFrame(columns = column_names) 

    nodes_col = ['Node','O_name','N_name','N_type', 'N_class', 'N_pack','args','ver'] 
    all_nodes = pd.DataFrame(columns = nodes_col) 
    all_nodes_e = pd.DataFrame(columns = nodes_col) 
    t = 1 
    for in_file in fileList: 
        #print(in_file) 
        ##print(\"iteration\",t) 
        t = t+1 
        f_name = (os.path.join(dot_path,in_file)) 
        ##print(f_name) 
        #f_name = (os.path.join(C_path,in_file)) 
        #f_name = (os.path.join(I_path,in_file)) 
        st = in_file.find(".dot") 
        csv_name = in_file[:st] 
        #print('csv_name',csv_name) 
        ##print(f_name) 

        edges_list = [] 
        Nodes_list = [] 

        lines = [ line.strip() for line in open(f_name)] 
        for i,line in enumerate(lines):    
            if line.find("->") != -1: #get the edges 
            #                
                #print(i, "  edge   ", line) 
    #             #print(\"first if\") 
    #             #print(50*'<>') 
                #print('++++++++++++++++++++++++++++++++++') 
#                 #print('edge', line) 
                l = line 
                src_st= line.rfind(" ->") 
                src_node= line[:src_st] 
#                 #print('proper source:', src_node) 
                tar_st = line.rfind('>') 
                tar_node = line [src_st+4: line.find(' [')] 
#                 #print('Target node : ',tar_node) 
#                 #print('+++++++++++++++++++++++++++++++++') 
    #             #print(\"nodes array    >\" , Nodes_arr) 
                source_node, target_node, edge_attr, edge_color, edge_style, edge_label= get_edge(line) 
                edges_list.append([source_node, target_node, edge_attr, edge_color, edge_style, edge_label,ver]) 
# this way I get the nodes line no duplicates 
            if (line.find("->") == -1  and line.find("Node") !=-1): 
                #print ('Node    ' ,line) 
                node_no,O_name, node_name, node_class, node_pack,node_type,args = get_node(line) 
                #print( 40*'#') 
                Nodes_list.append([node_no, O_name, node_name, node_type,node_class, node_pack,args,ver]) 


  # Edges data frame      
        try: 
            edges = pd.DataFrame(edges_list) # get the edge list to a panda dataframe 
            edges.columns= ['Nsource','Ntarget','edge_attr','e_color', 'e_style', 'e_label','ver'] 
        except Exception: 
            pass 
#         #print(edges.head()) 

        try: 
            if len(Nodes_list)!= 0:  
                #print(Nodes_list) 
                Nodes = pd.DataFrame(Nodes_list)      
                Nodes.columns= ['Node','O_name','N_name','N_type', 'N_class', 'N_pack','args','ver'] 


#                 #print(Nodes.head()) 
            #Nodes.drop_duplicates(subset = 'Node', keep='first',inplace=True) 
                edges= edges.assign(source =" ") 
                edges= edges.assign(target =" ") 
                edges= edges.assign(s_o =" ") 
                edges= edges.assign(t_o =" ") 

            # get the node names in the edged file 
                for i, row in edges.iterrows(): 
                    source = Nodes[Nodes['Node']==row['Nsource']]['N_name'].values[0] 
                    target = Nodes[Nodes['Node']==row['Ntarget']]['N_name'].values[0] 
                    src_old = Nodes[Nodes['Node']==row['Nsource']]['O_name'].values[0] 
                    trgt_old = Nodes[Nodes['Node']==row['Ntarget']]['O_name'].values[0] 
                    row["source"]=source 
                    row["target"]=target 
                    row["s_o"]=src_old 
                    row["t_o"]=trgt_old

        except Exception: 
            pass  

#         #print(edges.head()) 
        try: 
            if len(Nodes) !=0: 
                edges= edges[['source','target','s_o','t_o','edge_attr','e_color', 'e_style',
                              'e_label','Nsource','Ntarget','ver']] 
            #edges= edges[['source','target','edge_attr','Nsource','Ntarget']] 
                all_edges =  all_edges.append(edges,ignore_index = True) 
        except Exception: 
            pass 

        #print(csv_graph+csv_name+'.csv') 
        edges.to_csv(csv_graph+ csv_name+'_fixed_mm.csv', encoding='utf-8', index=False) 
#         Nodes.to_csv(csv_node+ csv_name+'_fixed_mm.csv', encoding = 'utf-8') 
        if len(Nodes)!= 0:   
            all_nodes = all_nodes.append(Nodes,ignore_index = True) 

            all_nodes_e = all_nodes.reset_index().drop_duplicates(subset = 'N_name',keep='first').set_index('index') 
            all_nodes_e = all_nodes_e.reset_index(drop=True) 
            #         # ADDING THE NODE ID COLUMN 
            all_nodes_e['Node_id'] = np.arange(len(all_nodes_e)) 
            all_nodes_e = all_nodes_e[['Node_id','Node','O_name','N_name','N_type', 'N_class', 'N_pack','args','ver']] 
    #     ############################################################################### 

    all_edges= all_edges[['source','target','s_o', 't_o','edge_attr','e_color', 'e_style', 'e_label','Nsource','Ntarget','ver']] 
    all_nodes_e.to_csv(csv_node+'Nodes_fixed.csv',encoding = 'utf-8',index = False)  
    print('file saves',csv_node+'Nodes_fixed.csv') 
    all_edges.to_csv(csv_graph+'all_fixed.csv',encoding='utf-8', index=False) 

# The main part calling get_network

In [105]:
import itertools 
for ver in soft_files:  
    print(ver)  

    soft_e = ver.rfind("/")  
    soft_n = ver[:soft_e]  
    soft_n1 = soft_n.rfind("/") +1  
    soft_name = ver[soft_n1:soft_e] +"/"  
    #         print(soft_name)  
    #os.makedirs(csv_path, ver)   
    ver_list = os.listdir(ver)  
    #print(\"ver list:   \  ver_list)  
    #     print(\"VER list   :\  ver_list)  
    #     print(50*'#')  
    ver_files = [(os.path.join(ver,infile )+"/")for infile in ver_list]  
    for dot_graph in ver_files:  
        print("version",  dot_graph)  
        dot_csv = dot_graph.rfind('_')  
    #             print(dot_csv)  
        end_graph = dot_graph.rfind('/')  
        ver_no = dot_graph[dot_csv+1: end_graph]  

        #print('version no:  ', ver_no)  
        dot_list = os.listdir(dot_graph)  

        graph_files = [(os.path.join(dot_graph,i )+"/")for i in dot_list]  


    #             print("dot graph:", dot_graph)  
        ver_loc = dot_graph.rfind("/")  

        #print(ver_loc)  
        ver_t = dot_graph[0:end_graph]  

        ver_loc_2 = ver_t.rfind("/")  
        ver_t2 = ver_t[ver_loc_2+1:]  
        ver_t2= ver_t2 +"/"  
    #              print( "VER_T2:" ,ver_t2)  
    #              print(\"DOT_list:", dot_list)  

        csv_path_files = [os.path.join(csv_path,soft_name,ver_t2,i)+"_csv/" for i in dot_list]  
#          print("CSV path    :",csv_path_files)  
        csv_nodes = [os.path.join(csv_path,soft_name,ver_t2,i)+"_nodes/" for i in dot_list]  
    #         print(\"CSV nodes:\ csv_nodes)  
    #         print(50*'#')  
        net_work = [os.makedirs(g,exist_ok=True) for g in csv_path_files]  


        net_nodes = [os.makedirs(n,exist_ok=True) for n in csv_nodes ]  
        if net_work ==[]:    
            net_work = csv_path_files 
        if net_nodes == None: 
            net_nodes = csv_nodes

#     #         print(\"DOT gratph : \", dot_graph) 
#     #         print(\"CSV path    :\",csv_path_files) 
#     #         print(\"nodes path \", csv_nodes) 



            
        for (a, b, c) in zip(graph_files, csv_path_files, csv_nodes):  
            print("a,b,c,ver:", a,b,c,ver_no) 
            get_network_fixed(a,b,c,ver_no) 



    

soft_files D:/mm_thesis/the_files/Data/Cassandra/
list ['Cassandra_030', 'Cassandra_040', 'Cassandra_041', 'Cassandra_042', 'Cassandra_050', 'Cassandra_051', 'Cassandra_060', 'Cassandra_061', 'Cassandra_0610', 'Cassandra_0611', 'Cassandra_0612', 'Cassandra_0613', 'Cassandra_062', 'Cassandra_063', 'Cassandra_064', 'Cassandra_065', 'Cassandra_066', 'Cassandra_067', 'Cassandra_068', 'Cassandra_069', 'Cassandra_070', 'Cassandra_071', 'Cassandra_0710', 'Cassandra_072', 'Cassandra_073', 'Cassandra_074', 'Cassandra_075', 'Cassandra_076', 'Cassandra_0762', 'Cassandra_077', 'Cassandra_078', 'Cassandra_079', 'Cassandra_080', 'Cassandra_081', 'Cassandra_0810', 'Cassandra_082', 'Cassandra_083', 'Cassandra_084', 'Cassandra_085', 'Cassandra_086', 'Cassandra_087', 'Cassandra_088', 'Cassandra_089', 'Cassandra_100', 'Cassandra_101', 'Cassandra_1010', 'Cassandra_1011', 'Cassandra_1012', 'Cassandra_102', 'Cassandra_103', 'Cassandra_104', 'Cassandra_105', 'Cassandra_106', 'Cassandra_107', 'Cassandra_108',

# getting nodes id in the edges file for source and target nodes
by getting the name of source & target from the edges file and matching them to the nodes file getting the Node_id 

# redo so you get the new files created 

In [ ]:
csv_path = 'D:/mm_thesis/Software_graphs/D_graph/'
csv_path_l = os.listdir(csv_path)
csv_files = [(os.path.join(csv_path,infile)+"/") for infile in csv_path_l]

In [ ]:
for net in csv_files:
    
   
    net_list = os.listdir(net)
    net_files = [(os.path.join(net,infile )+"/")for infile in net_list if os.path.isdir(os.path.join(net,infile))]
    
    
    #print(net_files)
    for net_graph in net_files:
        
        nn_list = os.listdir(net_graph)
        #print(nn_list)
        net_node_path = [(os.path.join(net_graph,i )+"/")for i in nn_list]
        #print(net_node_path)
        res = [[i , j] for i, j in zip(net_node_path[::2], net_node_path[1::2])] 
        for n in res:
            print(n[0])
            tt1 = os.listdir(n[0])      
            ttl =[i for i in tt1 if i.find('all') != -1]
            tt = ttl[0]
            file_net_work =(os.path.join(n[0]+tt))
            print("file edges", file_net_work)
            print(30 * '^')

            print(n)
            tn1 = os.listdir(n[1])
            #     print(len(tn1))
            #     print(n[1])
            tnl = [i for i in tn1 if i.find('Nodes_fixed') != -1]
            tn = tnl[0]
            file_node = os.path.join(n[1]+tn)
            print("file nodes",file_node,'\n')
            print(30 * '_')
            
            
            net_df = pd.read_csv(file_net_work)
            node_df = pd.read_csv(file_node)
        
            net_df= net_df.assign(s_id =" ")
            net_df= net_df.assign(t_id =" ")
            
#             print(net_df.head(2))
#             print(30*'^')
            for i, row in net_df.iterrows():
                
                
                s_i =node_df[node_df['N_name']==row['source']]['Node_id'].values[0]
                t_i =node_df[node_df['N_name']==row['target']]['Node_id'].values[0]
                net_df.at[i,"t_id"] = t_i
                net_df.at[i,"s_id"] = s_i
#                 row['t_id'] = t_i
#                 row['s_id'] = s_i
#                 print(row)
                
#             print(net_df.head(2))
#             print("net col BEFORE update", net_df.columns)
            
            net_df.rename(columns= {'source':'o_source', 'target' :'o_target','s_id': 'source', 't_id' : 'target'}, inplace = True)
            net_df = net_df[['source','target','o_source','o_target','s_o','t_o','edge_attr','e_color', 'e_style',
                              'e_label','Nsource','Ntarget','ver']]         
#             print("net col after update", net_df.columns)
#             print(net_df.head(2))
#             print(30* '*')
            net_df.to_csv(n[0]+"edges.csv", encoding='utf-8',index = False)

# adding the time stamp


In [29]:
csv_path = 'D:/mm_thesis/Software_graphs/D_graph/'
csv_path_l = os.listdir(csv_path)
csv_files1 = [(os.path.join(csv_path,infile)+"/") for infile in csv_path_l if os.path.isdir(os.path.join(csv_path,infile))]
csv_files = [i for i in csv_files1 if i.find("Time") == -1]
look_path = 'D:/mm_thesis/Software_graphs/lookup.csv'

In [30]:
Lookup_df = pd.read_csv(look_path)
Lookup_df.tail(50)

,Unnamed: 0,tarfile,netfile,Timestamp,Time_created,yaml,stamp,d
97,97,ant-ivy-2.1.0-rc1.tar.gz,ant_ivy_210.1,1238452019,2009-03-31-00:26,ivy210rc1,1238450400,31-Mar-09
98,98,ant-ivy-2.2.0.tar.gz,ant_ivy_220,1285278809,2010-09-23-23:53,ivy220,1285192800,23-Sep-10
99,99,ant-ivy-2.2.0-rc1.tar.gz,ant_ivy_220.1,1277929768,2010-06-30-22:29,ivy220rc1,1277848800,30-Jun-10
100,100,ant-ivy-2.3.0.tar.gz,ant_ivy_230,1358799148,2013-01-21-21:12,ivy230,1357772400,10-Jan-13
101,101,ant-ivy-2.3.0-rc1.tar.gz,ant_ivy_230.1,1334530880,2012-04-16-01:01,ivy230rc1,1334440800,15-Apr-12
102,102,ant-ivy-2.3.0-rc2.tar.gz,ant_ivy_230.2,1352154854,2012-11-05-23:34,ivy230rc2,1352070000,5-Nov-12
103,103,ant-ivy-2.4.0.tar.gz,ant_ivy_240,1418486302,2014-12-13-16:58,ivy240,1418425200,13-Dec-14
104,104,ant-ivy-2.4.0-rc1.tar.gz,ant_ivy_240.1,1394921663,2014-03-15-23:14,ivy240rc1,1394838000,15-Mar-14
105,105,ant-ivy-2.4.0-rc1_OLD.tar.gz,ant_ivy_240.01,1394921584,2014-03-15-23:13,ivy240rc1old,1390172400,20-Jan-14
106,106,httpcomponents-client-4.0.2-RC1.tar.gz,HC_402.1,1283080710,2010-08-29-13:18,http402rc1,1283032800,29-Aug-10


In [31]:
net_csv = csv_files[5:6]

In [32]:
net_csv
for net in net_csv:
    print(net)


D:/mm_thesis/D_graph/Ivy/


In [33]:
for net in net_csv:
#  for net in csv_files:
    net_list = os.listdir(net)
    net_files1 = [(os.path.join(net,infile )+"/")for infile in net_list if os.path.isdir(os.path.join(net,infile))]
    net_files =  [ i for i in net_files1 if (i.find('time')==-1)]
#     print('net_files  :',net_files)
    #print(net_files)
    for net_graph in net_files:
        print(net_graph)
        ver_lt = net_graph[:-1] #just remove / in the end
        ver_st = ver_lt.rfind('/')
        ver_no = ver_lt[ver_st+1:]
        print(ver_no)
#          Get the info from the lookup file 
#         print( Lookup_df[Lookup_df['netfile'] == ver_no])
        info_ver = Lookup_df[Lookup_df['netfile'] == ver_no]
        tar_name = Lookup_df[Lookup_df['netfile'] == ver_no]['tarfile'].values[0]
        t_stmp = Lookup_df[Lookup_df['netfile'] == ver_no]['stamp'].values[0]
      
            
        ver_end = tar_name.find('.tar')
        Version = tar_name[:ver_end]
        print(30*'___')
        print('to be save', Version,'-->', t_stmp)
        print(30*'___')
        nn_list = os.listdir(net_graph)
        #print(nn_list)
        c_path = [(os.path.join(net_graph,i )+"/")for i in nn_list if os.path.isdir(os.path.join(net_graph,i))]
#       
        print(30*'#')
        
#         print('c_path  :', c_path)
        for a_dir in c_path:
#             print(a_dir)
            adir_list1 = [ i for i in os.listdir(a_dir) if (i.find('fixed.csv') != -1 or i.find('edges') != -1)]
            adir_list = [i for i in adir_list1 if i.find('Ver_stmp') ==-1 ]
#             print('list1:  ',adir_list1)
#             print('list:      ', adir_list)
            for fl in adir_list:
#                 print(fl)
                save_name= "Ver_stmp_"+fl
                to_save = os.path.join(a_dir,save_name)

                f_read = a_dir+ fl 
#                 print('to read', f_read)
                df= pd.read_csv(f_read)
#                 df.insert(loc = 0, column = 'Version', value = Version)
#                 df.insert(loc = 1, column = 't_stmp', value = t_stmp)
                                                          
                df['Version'] = Version
                df['t_stmp'] = t_stmp
#                 print(df.head(2))
                print('to_save', to_save)
#                                                 
                df.to_csv(to_save,encoding='utf-8',index = False)
                                    

D:/mm_thesis/D_graph/Ivy/ant_ivy_141/
ant_ivy_141
__________________________________________________________________________________________
to be save ant-ivy-1.4.1 --> 1165705200
__________________________________________________________________________________________
##############################
to_save D:/mm_thesis/D_graph/Ivy/ant_ivy_141/CDot_csv/Ver_stmp_all_fixed.csv
to_save D:/mm_thesis/D_graph/Ivy/ant_ivy_141/CDot_csv/Ver_stmp_edges.csv
to_save D:/mm_thesis/D_graph/Ivy/ant_ivy_141/CDot_nodes/Ver_stmp_Nodes_fixed.csv
to_save D:/mm_thesis/D_graph/Ivy/ant_ivy_141/collDot_csv/Ver_stmp_all_fixed.csv
to_save D:/mm_thesis/D_graph/Ivy/ant_ivy_141/collDot_csv/Ver_stmp_edges.csv
to_save D:/mm_thesis/D_graph/Ivy/ant_ivy_141/collDot_nodes/Ver_stmp_Nodes_fixed.csv
to_save D:/mm_thesis/D_graph/Ivy/ant_ivy_141/InhDot_csv/Ver_stmp_all_fixed.csv
to_save D:/mm_thesis/D_graph/Ivy/ant_ivy_141/InhDot_csv/Ver_stmp_edges.csv
to_save D:/mm_thesis/D_graph/Ivy/ant_ivy_141/InhDot_nodes/Ver_stmp_Nodes

to_save D:/mm_thesis/D_graph/Ivy/ant_ivy_210.1/CDot_csv/Ver_stmp_edges.csv
to_save D:/mm_thesis/D_graph/Ivy/ant_ivy_210.1/CDot_nodes/Ver_stmp_Nodes_fixed.csv
to_save D:/mm_thesis/D_graph/Ivy/ant_ivy_210.1/CollDot_csv/Ver_stmp_all_fixed.csv
to_save D:/mm_thesis/D_graph/Ivy/ant_ivy_210.1/CollDot_csv/Ver_stmp_edges.csv
to_save D:/mm_thesis/D_graph/Ivy/ant_ivy_210.1/CollDot_nodes/Ver_stmp_Nodes_fixed.csv
to_save D:/mm_thesis/D_graph/Ivy/ant_ivy_210.1/InhDot_csv/Ver_stmp_all_fixed.csv
to_save D:/mm_thesis/D_graph/Ivy/ant_ivy_210.1/InhDot_csv/Ver_stmp_edges.csv
to_save D:/mm_thesis/D_graph/Ivy/ant_ivy_210.1/InhDot_nodes/Ver_stmp_Nodes_fixed.csv
D:/mm_thesis/D_graph/Ivy/ant_ivy_210.2/
ant_ivy_210.2
__________________________________________________________________________________________
to be save ant-ivy-2.1.0-rc2 --> 1246658400
__________________________________________________________________________________________
##############################
to_save D:/mm_thesis/D_graph/Ivy/ant_ivy_2

to_save D:/mm_thesis/D_graph/Ivy/ant_ivy_240.01/CDot_csv/Ver_stmp_edges.csv
to_save D:/mm_thesis/D_graph/Ivy/ant_ivy_240.01/CDot_nodes/Ver_stmp_Nodes_fixed.csv
to_save D:/mm_thesis/D_graph/Ivy/ant_ivy_240.01/CollDot_csv/Ver_stmp_all_fixed.csv
to_save D:/mm_thesis/D_graph/Ivy/ant_ivy_240.01/CollDot_csv/Ver_stmp_edges.csv
to_save D:/mm_thesis/D_graph/Ivy/ant_ivy_240.01/CollDot_nodes/Ver_stmp_Nodes_fixed.csv
to_save D:/mm_thesis/D_graph/Ivy/ant_ivy_240.01/InhDot_csv/Ver_stmp_all_fixed.csv
to_save D:/mm_thesis/D_graph/Ivy/ant_ivy_240.01/InhDot_csv/Ver_stmp_edges.csv
to_save D:/mm_thesis/D_graph/Ivy/ant_ivy_240.01/InhDot_nodes/Ver_stmp_Nodes_fixed.csv
D:/mm_thesis/D_graph/Ivy/ant_ivy_240.1/
ant_ivy_240.1
__________________________________________________________________________________________
to be save ant-ivy-2.4.0-rc1 --> 1394838000
__________________________________________________________________________________________
##############################
to_save D:/mm_thesis/D_graph/Ivy/a

--

# creating the txt files to be used in matlab, for other network metrics

In [42]:
from pathlib import Path
import os.path
import os
import numpy as np
import re
import pandas as pd
import shutil 
import networkx as nx
import dynetx as dn
import powerlaw
#from networkx.algorithms import community
import networkx.algorithms.community as nxcom
import community# as community_louvain
import matplotlib.pyplot as plt

In [43]:
csv_path =  'D:/mm_thesis/Software_graphs/D_graph/'
softwareList = os.listdir(soft_path)
soft_files = [(os.path.join(soft_path,infile)+"/") for infile in softwareList]
csv_path_l = os.listdir(csv_path)
csv_files = [(os.path.join(csv_path,infile)+"/") for infile in csv_path_l if os.path.isdir(os.path.join(csv_path,infile))]


In [36]:
csv_files

['D:/mm_thesis/D_graph/all_metrics/',
 'D:/mm_thesis/D_graph/Cassandra/',
 'D:/mm_thesis/D_graph/Chukwa/',
 'D:/mm_thesis/D_graph/D_matlab/',
 'D:/mm_thesis/D_graph/httpClient/',
 'D:/mm_thesis/D_graph/Ivy/',
 'D:/mm_thesis/D_graph/Jena/',
 'D:/mm_thesis/D_graph/New folder/',
 'D:/mm_thesis/D_graph/Time_stamps/']

In [37]:
dot_file = ['collDot_time', 'InhDot_time' , 'CDot_time']

In [33]:
for i in csv_files:
    print(i)
    for g in dot_file:
        print(g)
        g_path = os.path.join(i,g)+ '/'
        print(g_path)
        os.makedirs(g_path,exist_ok=True)


D:/mm_thesis/D_graph/Cassandra/
collDot_time
D:/mm_thesis/D_graph/Cassandra/collDot_time/
InhDot_time
D:/mm_thesis/D_graph/Cassandra/InhDot_time/
CDot_time
D:/mm_thesis/D_graph/Cassandra/CDot_time/
D:/mm_thesis/D_graph/Chukwa/
collDot_time
D:/mm_thesis/D_graph/Chukwa/collDot_time/
InhDot_time
D:/mm_thesis/D_graph/Chukwa/InhDot_time/
CDot_time
D:/mm_thesis/D_graph/Chukwa/CDot_time/
D:/mm_thesis/D_graph/httpClient/
collDot_time
D:/mm_thesis/D_graph/httpClient/collDot_time/
InhDot_time
D:/mm_thesis/D_graph/httpClient/InhDot_time/
CDot_time
D:/mm_thesis/D_graph/httpClient/CDot_time/
D:/mm_thesis/D_graph/Ivy/
collDot_time
D:/mm_thesis/D_graph/Ivy/collDot_time/
InhDot_time
D:/mm_thesis/D_graph/Ivy/InhDot_time/
CDot_time
D:/mm_thesis/D_graph/Ivy/CDot_time/
D:/mm_thesis/D_graph/Jena/
collDot_time
D:/mm_thesis/D_graph/Jena/collDot_time/
InhDot_time
D:/mm_thesis/D_graph/Jena/InhDot_time/
CDot_time
D:/mm_thesis/D_graph/Jena/CDot_time/
D:/mm_thesis/D_graph/Time_stamps/
collDot_time
D:/mm_thesis/D_

In [44]:
# net_csv = csv_files[2:3]
csv_net = csv_files[5:6]
print(csv_net)

['D:/mm_thesis/D_graph/Ivy/']


In [45]:
for i in csv_files:
    print(i)

D:/mm_thesis/D_graph/all_metrics/
D:/mm_thesis/D_graph/Cassandra/
D:/mm_thesis/D_graph/Chukwa/
D:/mm_thesis/D_graph/D_matlab/
D:/mm_thesis/D_graph/httpClient/
D:/mm_thesis/D_graph/Ivy/
D:/mm_thesis/D_graph/Jena/
D:/mm_thesis/D_graph/New folder/
D:/mm_thesis/D_graph/Time_stamps/


In [48]:

dot_types = ['CDot_', 'CollDot_', 'InhDot_']
# dot_types = ['CDot_csv', 'CollDot_csv', 'InhDot_csv']

file_name = 'Ver_stmp_edges.csv'
for net in csv_net:
# for net in csv_files:
    print(net)
    net_list = os.listdir(net)
    net_files = [(os.path.join(net,infile )+"/")for infile in net_list if os.path.isdir(os.path.join(net,infile))]
#     print(net_files)
    net_edges = [i for i in net_files if i.find('_time/') == -1]
#     print(net_edges)
    for d in dot_types:
        print('type: ', d)
        d_type = d+'csv'
        dot_path = [os.path.join(i,d_type) for i in net_edges]
        print('dot path :', dot_path)
        t_type = d + 'time'
        time_path = os.path.join(net,t_type)+'/'
        print('time_path   ', time_path)
        icount = 0
        for dfile in dot_path:
            icount = icount + 1
            edge_file  =  os.path.join(dfile,file_name)
            print('edge file:' , edge_file)
            edge_df = pd.read_csv(edge_file)
            e = edge_df[edge_df[['source', 'target']].duplicated() == False]
            x_t = edge_df['t_stmp'].tolist()[0]
            print(x_t)
            gt = nx.from_pandas_edgelist(e, source = 'source', target='target',edge_attr=True, create_using=nx.DiGraph())
        
            x_t = str(x_t)
            x_name = x_t+'.txt'
            print('time - path to make sure', time_path)
            x_file = os.path.join(time_path,x_name)
            print()
            tst =[e for e in gt.edges]
            print(len(tst))
            s_e =list( gt.edges)
            print(len(s_e))
            print(x_file)
            df = pd.DataFrame(s_e, columns =['source', 'target'])
            df.to_csv(x_file, sep='\t', index=False, header = False)
        print(' number of file :',icount)

            
    
    

D:/mm_thesis/D_graph/Ivy/
type:  CDot_
dot path : ['D:/mm_thesis/D_graph/Ivy/ant_ivy_141/CDot_csv', 'D:/mm_thesis/D_graph/Ivy/ant_ivy_200/CDot_csv', 'D:/mm_thesis/D_graph/Ivy/ant_ivy_200.1/CDot_csv', 'D:/mm_thesis/D_graph/Ivy/ant_ivy_200.12/CDot_csv', 'D:/mm_thesis/D_graph/Ivy/ant_ivy_200.2/CDot_csv', 'D:/mm_thesis/D_graph/Ivy/ant_ivy_200.21/CDot_csv', 'D:/mm_thesis/D_graph/Ivy/ant_ivy_200.22/CDot_csv', 'D:/mm_thesis/D_graph/Ivy/ant_ivy_210/CDot_csv', 'D:/mm_thesis/D_graph/Ivy/ant_ivy_210.1/CDot_csv', 'D:/mm_thesis/D_graph/Ivy/ant_ivy_210.2/CDot_csv', 'D:/mm_thesis/D_graph/Ivy/ant_ivy_220/CDot_csv', 'D:/mm_thesis/D_graph/Ivy/ant_ivy_220.1/CDot_csv', 'D:/mm_thesis/D_graph/Ivy/ant_ivy_230/CDot_csv', 'D:/mm_thesis/D_graph/Ivy/ant_ivy_230.1/CDot_csv', 'D:/mm_thesis/D_graph/Ivy/ant_ivy_230.2/CDot_csv', 'D:/mm_thesis/D_graph/Ivy/ant_ivy_240/CDot_csv', 'D:/mm_thesis/D_graph/Ivy/ant_ivy_240.01/CDot_csv', 'D:/mm_thesis/D_graph/Ivy/ant_ivy_240.1/CDot_csv']
time_path    D:/mm_thesis/D_graph/Ivy/C

1277848800
time - path to make sure D:/mm_thesis/D_graph/Ivy/CollDot_time/

667
667
D:/mm_thesis/D_graph/Ivy/CollDot_time/1277848800.txt
edge file: D:/mm_thesis/D_graph/Ivy/ant_ivy_230/CollDot_csv\Ver_stmp_edges.csv
1357772400
time - path to make sure D:/mm_thesis/D_graph/Ivy/CollDot_time/

868
868
D:/mm_thesis/D_graph/Ivy/CollDot_time/1357772400.txt
edge file: D:/mm_thesis/D_graph/Ivy/ant_ivy_230.1/CollDot_csv\Ver_stmp_edges.csv
1334440800
time - path to make sure D:/mm_thesis/D_graph/Ivy/CollDot_time/

866
866
D:/mm_thesis/D_graph/Ivy/CollDot_time/1334440800.txt
edge file: D:/mm_thesis/D_graph/Ivy/ant_ivy_230.2/CollDot_csv\Ver_stmp_edges.csv
1352070000
time - path to make sure D:/mm_thesis/D_graph/Ivy/CollDot_time/

868
868
D:/mm_thesis/D_graph/Ivy/CollDot_time/1352070000.txt
edge file: D:/mm_thesis/D_graph/Ivy/ant_ivy_240/CollDot_csv\Ver_stmp_edges.csv
1418425200
time - path to make sure D:/mm_thesis/D_graph/Ivy/CollDot_time/

934
934
D:/mm_thesis/D_graph/Ivy/CollDot_time/1418425200

# for showing a net work 

In [16]:
    # disp_path = 'C:/Users/mmata/Desktop/display_network_cbuild/' 
    # dot_graph =  'C:/Users/mmata/Desktop/display_network_cbuild/dot_files/' 
    # soft_files ='C:/Users/mmata/Desktop/display_network_cbuild/disp_cdot/' 
    disp_path = 'D:/mm_thesis/display_network_cbuild/' 
    dot_graph =  'D:/mm_thesis/display_network_cbuild/dot_files/' 
    soft_files ='D:/mm_thesis/display_network_cbuild/disp_cdot/' 
    csv_path = os.listdir(soft_files) 
    dot_lis = os.listdir(dot_graph) 
    # soft_files = [(os.path.join(disp_path,infile)+\"/\") for infile in softwareList]"


In [17]:
print(dot_graph) 
print(soft_files) 
print(csv_path[0]) 


D:/mm_thesis/display_network_cbuild/dot_files/
D:/mm_thesis/display_network_cbuild/disp_cdot/
CDot_csv


In [18]:

graph_files = [(os.path.join(dot_graph,i ))for i in dot_lis]
print(graph_files) 


['D:/mm_thesis/display_network_cbuild/dot_files/classorg_1_1apache_1_1cassandra_1_1analytics_1_1_analytics_context_a0e8f5833d5e9e1d49282e000a4f03d6b_cgraph.dot', 'D:/mm_thesis/display_network_cbuild/dot_files/classorg_1_1apache_1_1cassandra_1_1analytics_1_1_analytics_context_a2b68f0acd21b145e9dc2d9b90ea7a375_cgraph.dot']


In [22]:
dot_lis =  os.listdir(dot_graph) 
graph_files = [(os.path.join(dot_graph,i ))for i in dot_lis] 
# print(graph_files) 
ver_t2 = 'v1' 
csv_path_files = os.path.join(soft_files,csv_path[0]+"/") 
# print("CSV path    :",csv_path_files) 
csv_nodes = os.path.join(soft_files,csv_path[1]+'/') 
# print("nodes",csv_nodes) 

for i in graph_files: 
    grt_network_nover_fixed(dot_graph,csv_path_files, csv_nodes) 

    # for (a, b, c) in zip(graph_files, csv_path_files, csv_nodes):  
    #     print("a,b,c,ver   :", a,b,c) 
    #     get_network_nover(a,b,c) 


classorg_1_1apache_1_1cassandra_1_1analytics_1_1_analytics_context_a0e8f5833d5e9e1d49282e000a4f03d6b_cgraph.dot
csv_name classorg_1_1apache_1_1cassandra_1_1analytics_1_1_analytics_context_a0e8f5833d5e9e1d49282e000a4f03d6b_cgraph
Node     Node1 [label="org.apache.cassandra.analytics.\lAnalyticsContext.emitMetric",height=0.2,width=0.4,color="black", fillcolor="grey75", style="filled", fontcolor="black",tooltip=" "];
########################################
7   edge    Node1 -> Node2 [color="midnightblue",fontsize="10",style="solid",fontname="Helvetica"];
++++++++++++++++++++++++++++++++++
Node     Node2 [label="org.apache.cassandra.analytics.\lAnalyticsContext.getDmax",height=0.2,width=0.4,color="black", fillcolor="white", style="filled",URL="$classorg_1_1apache_1_1cassandra_1_1analytics_1_1_analytics_context.html#a1c13dfc9111befc9805a6a072413928d",tooltip=" "];
########################################
9   edge    Node1 -> Node3 [color="midnightblue",fontsize="10",style="solid",fontname=

C:\Users\mmata\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


classorg_1_1apache_1_1cassandra_1_1analytics_1_1_analytics_context_a2b68f0acd21b145e9dc2d9b90ea7a375_cgraph.dot
csv_name classorg_1_1apache_1_1cassandra_1_1analytics_1_1_analytics_context_a2b68f0acd21b145e9dc2d9b90ea7a375_cgraph
Node     Node1 [label="org.apache.cassandra.analytics.\lAnalyticsContext.emitRecord",height=0.2,width=0.4,color="black", fillcolor="grey75", style="filled", fontcolor="black",tooltip=" "];
########################################
7   edge    Node1 -> Node2 [color="midnightblue",fontsize="10",style="solid",fontname="Helvetica"];
++++++++++++++++++++++++++++++++++
Node     Node2 [label="org.apache.cassandra.analytics.\lAnalyticsContext.emitMetric",height=0.2,width=0.4,color="black", fillcolor="white", style="filled",URL="$classorg_1_1apache_1_1cassandra_1_1analytics_1_1_analytics_context.html#a0e8f5833d5e9e1d49282e000a4f03d6b",tooltip=" "];
########################################
9   edge    Node2 -> Node3 [color="midnightblue",fontsize="10",style="solid",fontna

In [21]:
def grt_network_nover_fixed(dot_path, csv_graph,csv_node): 
    fileList = os.listdir(dot_path) 
    dot_files = [os.path.join(dot_path,infile )for infile in fileList] 

    column_names = ['Nsource','Ntarget','edge_attr', 'e_color', 'e_style', 'e_label', 'source', 'target'] 
    #umn_names = ['source', 'target','edge_attr','Nsource','Ntarget'] 
    all_edges = pd.DataFrame(columns = column_names) 

    nodes_col = ['Node','O_name','N_name','N_type', 'N_class', 'N_pack','args'] 
    all_nodes = pd.DataFrame(columns = nodes_col) 
    all_nodes_e = pd.DataFrame(columns = nodes_col) 
    t = 1 
    for in_file in fileList: 
        print(in_file) 
        #print(\"iteration\",t) 
        t = t+1 
        f_name = (os.path.join(dot_path,in_file)) 
        #print(f_name) 
        #f_name = (os.path.join(C_path,in_file)) 
        #f_name = (os.path.join(I_path,in_file)) 
        st = in_file.find(".dot") 
        csv_name = in_file[:st] 
        print('csv_name',csv_name) 
        #print(f_name) 

        edges_list = [] 
        Nodes_list = [] 

        lines = [ line.strip() for line in open(f_name)] 
        for i,line in enumerate(lines):    
            if line.find("->") != -1: #get the edges 
            #                
                print(i, "  edge   ", line) 
    #             print(\"first if\") 
    #             print(50*'<>') 
                print('++++++++++++++++++++++++++++++++++') 
#                 print('edge', line) 
                l = line 
                src_st= line.rfind(" ->") 
                src_node= line[:src_st] 
#                 print('proper source:', src_node) 
                tar_st = line.rfind('>') 
                tar_node = line [src_st+4: line.find(' [')] 
#                 print('Target node : ',tar_node) 
#                 print('+++++++++++++++++++++++++++++++++') 
    #             print(\"nodes array    >\" , Nodes_arr) 
                source_node, target_node, edge_attr, edge_color, edge_style, edge_label= get_edge(line) 
                edges_list.append([source_node, target_node, edge_attr, edge_color, edge_style, edge_label]) 
# this way I get the nodes line no duplicates 
            if (line.find("->") == -1  and line.find("Node") !=-1): 
                print ('Node    ' ,line) 
                node_no,O_name, node_name, node_class, node_pack,node_type,args = get_node(line) 
                print( 40*'#') 
                Nodes_list.append([node_no, O_name, node_name, node_type,node_class, node_pack,args]) 


  # Edges data frame      
        try: 
            edges = pd.DataFrame(edges_list) # get the edge list to a panda dataframe 
            edges.columns= ['Nsource','Ntarget','edge_attr','e_color', 'e_style', 'e_label'] 
        except Exception: 
            pass 
#         print(edges.head()) 

        try: 
            if len(Nodes_list)!= 0:  
                print(Nodes_list) 
                Nodes = pd.DataFrame(Nodes_list)      
                Nodes.columns= ['Node','O_name','N_name','N_type', 'N_class', 'N_pack','args'] 


#                 print(Nodes.head()) 
            #Nodes.drop_duplicates(subset = 'Node', keep='first',inplace=True) 
                edges= edges.assign(source =" ") 
                edges= edges.assign(target =" ") 
                edges= edges.assign(s_o =" ") 
                edges= edges.assign(t_o =" ") 

            # get the node names in the edged file 
                for i, row in edges.iterrows(): 
                    source = Nodes[Nodes['Node']==row['Nsource']]['N_name'].values[0] 
                    target = Nodes[Nodes['Node']==row['Ntarget']]['N_name'].values[0] 
                    src_old = Nodes[Nodes['Node']==row['Nsource']]['O_name'].values[0] 
                    trgt_old = Nodes[Nodes['Node']==row['Ntarget']]['O_name'].values[0] 
                    row["source"]=source 
                    row["target"]=target 
                    row["s_o"]=source 
                    row["t_o"]=target 

        except Exception: 
            pass  

#         print(edges.head()) 
        try: 
            if len(Nodes) !=0: 
                edges= edges[['source','target','s_o','t_o','edge_attr','e_color', 'e_style', 'e_label','Nsource','Ntarget']] 
            #edges= edges[['source','target','edge_attr','Nsource','Ntarget']] 
                all_edges =  all_edges.append(edges,ignore_index = True) 
        except Exception: 
            pass 

        print(csv_graph+csv_name+'.csv') 
        edges.to_csv(csv_graph+ csv_name+'_fixed_mm.csv', encoding='utf-8', index=False) 
        Nodes.to_csv(csv_node+ csv_name+'_fixed_mm.csv', encoding = 'utf-8') 
        if len(Nodes)!= 0:   
            all_nodes = all_nodes.append(Nodes,ignore_index = True) 

            all_nodes_e = all_nodes.reset_index().drop_duplicates(subset = 'N_name',keep='first').set_index('index') 
            all_nodes_e = all_nodes_e.reset_index(drop=True) 
            #         # ADDING THE NODE ID COLUMN 
            all_nodes_e['Node_id'] = np.arange(len(all_nodes_e)) 
            all_nodes_e = all_nodes_e[['Node_id','Node','O_name','N_name','N_type', 'N_class', 'N_pack','args']] 
    #     ############################################################################### 

    all_edges= all_edges[['source','target','s_o', 't_o','edge_attr','e_color', 'e_style', 'e_label','Nsource','Ntarget']] 
    all_nodes_e.to_csv(csv_node+'Nodes_fixed.csv',encoding = 'utf-8')  
    print('file saves',csv_node+'Nodes_fixed.csv') 
    all_edges.to_csv(csv_graph+'all_fixed.csv',encoding='utf-8', index=False) 





        #         print(\"before\") 
#         print(Nodes_list) 
#         print(len(Nodes_list)) 
#         print(30*'<>') 
#         tpls = [tuple(x) for x in Nodes_list] 
#         dct = list(dict.fromkeys(tpls)) 
#         dup_free = [list(x) for x in Nodes_list] 
#         print(\"After\") 
#         print(dup_free) 
#         print(len(dup_free)) 
#         print(30*'#') 

#         Nodes = pd.DataFrame(Nodes_list)      
#         Nodes.columns= ['Node','O_name','N_name','N_type', 'N_class', 'N_pack','args'] 
#         #g3t 
# #     except Exception: 
# #         pass 



# deleted the Ver_stmp by hand for  this dir build it up again

In [251]:
net_graph ='D:/mm_thesis/Software_graphs/D_graph/Cassandra/Cassandra_201/'

In [257]:
print(net_graph)
ver_lt = net_graph[:-1] #just remove / in the end
ver_st = ver_lt.rfind('/')
ver_no = ver_lt[ver_st+1:]
print(ver_no)
info_ver = Lookup_df[Lookup_df['netfile'] == ver_no]
tar_name = Lookup_df[Lookup_df['netfile'] == ver_no]['tarfile'].values[0]
t_stmp = Lookup_df[Lookup_df['netfile'] == ver_no]['stamp'].values[0]
ver_end = tar_name.find('.tar')
Version = tar_name[:ver_end]
print('to be save', Version, t_stmp)
nn_list = os.listdir(net_graph)
c_path = [(os.path.join(net_graph,i )+"/")for i in nn_list if os.path.isdir(os.path.join(net_graph,i))]
for a_dir in c_path:
#             print(a_dir)
    adir_list = [ i for i in os.listdir(a_dir) if (i.find('fixed.csv') != -1 or i.find('edges') != -1)]
    print(adir_list)
    for fl in adir_list:
#                 print(fl)
        save_name= "Ver_stmp_"+fl
        to_save = os.path.join(a_dir,save_name)
        f_read = a_dir+ fl 
#                 print('to read', f_read)
        df= pd.read_csv(f_read)
#                 df.insert(loc = 0, column = 'Version', value = Version)
#                 df.insert(loc = 1, column = 't_stmp', value = t_stmp)

        df['Version'] = Version
        df['t_stmp'] = t_stmp
#                 print(df.head(2))
        print('to_save', to_save)
#                                                 
        df.to_csv(to_save,encoding='utf-8',index = False)



D:/mm_thesis/D_graph/Cassandra/Cassandra_201/
Cassandra_201
to be save cassandra-cassandra-2.0.1 1379628000
['all_fixed.csv', 'edges.csv']
to_save D:/mm_thesis/D_graph/Cassandra/Cassandra_201/CDot_csv/Ver_stmp_all_fixed.csv
to_save D:/mm_thesis/D_graph/Cassandra/Cassandra_201/CDot_csv/Ver_stmp_edges.csv
['Nodes_fixed.csv']
to_save D:/mm_thesis/D_graph/Cassandra/Cassandra_201/CDot_nodes/Ver_stmp_Nodes_fixed.csv
['all_fixed.csv', 'edges.csv']
to_save D:/mm_thesis/D_graph/Cassandra/Cassandra_201/CollDot_csv/Ver_stmp_all_fixed.csv
to_save D:/mm_thesis/D_graph/Cassandra/Cassandra_201/CollDot_csv/Ver_stmp_edges.csv
['Nodes_fixed.csv']
to_save D:/mm_thesis/D_graph/Cassandra/Cassandra_201/CollDot_nodes/Ver_stmp_Nodes_fixed.csv
['all_fixed.csv', 'edges.csv']
to_save D:/mm_thesis/D_graph/Cassandra/Cassandra_201/InhDot_csv/Ver_stmp_all_fixed.csv
to_save D:/mm_thesis/D_graph/Cassandra/Cassandra_201/InhDot_csv/Ver_stmp_edges.csv
['Nodes_fixed.csv']
to_save D:/mm_thesis/D_graph/Cassandra/Cassandra_2